# Bibliotecas Necessárias

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import re

# Leitura dos Dados

In [1]:
train_data = pd.read_csv("/kaggle/input/mini-flight-delay-prediction/flight_delays_train.csv")
train_data.head()

In [1]:
train_data.info()

In [1]:
test = pd.read_csv("/kaggle/input/mini-flight-delay-prediction/flight_delays_test.csv")
test.head()

In [1]:
test.info()

# Pré processamento dos dados

In [1]:
def monthDay2number(text):
    text = int(re.sub(r"c-", "", text))
    return text

In [1]:
train_data.Month = train_data.Month.map(lambda month: monthDay2number(month))
train_data.DayofMonth = train_data.DayofMonth.map(lambda day: monthDay2number(day))
train_data.DayOfWeek = train_data.DayOfWeek.map(lambda day: monthDay2number(day))
train_data.drop(columns=['Origin','Dest'], inplace=True)

train_data.dep_delayed_15min = train_data.dep_delayed_15min.map(lambda label: 0 if label=='Y' else 1)

In [1]:
test.Month = test.Month.map(lambda month: monthDay2number(month))
test.DayofMonth = test.DayofMonth.map(lambda day: monthDay2number(day))
test.DayOfWeek = test.DayOfWeek.map(lambda day: monthDay2number(day))
test.drop(columns=['Origin','Dest'], inplace=True)

In [1]:
train_data = pd.get_dummies(train_data)
test = pd.get_dummies(test)

for column in train_data.columns:
    if not column in test.columns:
        test[column] = pd.DataFrame(0, index=np.arange(len(test)), columns=[column])

for column in test.columns:
    if not column in train_data.columns:
        train_data[column] = pd.DataFrame(0, index=np.arange(len(train_data)), columns=[column])

# Escolha do Classificador

In [1]:
train, test = train_test_split(train_data)

In [1]:
labels_train = train.dep_delayed_15min
train.drop(columns="dep_delayed_15min", inplace=True)

labels_test = test.dep_delayed_15min
test.drop(columns="dep_delayed_15min", inplace=True)

### SVM

In [1]:
clf_svm = svm.SVC(class_weight='balanced')
clf_svm.fit(train, labels_train)

In [1]:
labels_predicted = clf_svm.predict(test)
print('F1-score: ', metrics.f1_score(labels_test, labels_predicted))

In [1]:
metrics.confusion_matrix(labels_test, labels_predicted)

### Random Forest

In [1]:
clf_randomforest = RandomForestClassifier(n_estimators=100, class_weight='balanced')
clf_randomforest.fit(train, labels_train)

In [1]:
labels_predicted = clf_randomforest.predict(test)
print('F1-score: ', metrics.f1_score(labels_test, labels_predicted))

In [1]:
metrics.confusion_matrix(labels_test, labels_predicted)

###  Gradient Boosting

In [1]:
clf_boosting =  GradientBoostingClassifier()
clf_boosting.fit(train, labels_train)

In [1]:
labels_predicted = clf_boosting.predict(test)
print('F1-score: ', metrics.f1_score(labels_test, labels_predicted))

In [1]:
metrics.confusion_matrix(labels_test, labels_predicted)

### MLP

In [1]:
clf_mlp = MLPClassifier(hidden_layer_sizes=(500, 20), max_iter=3000)
clf_mlp.fit(train, labels_train)

In [1]:
labels_predicted = clf_mlp.predict(test)
print('F1-score: ', metrics.f1_score(labels_test, labels_predicted))

In [1]:
metrics.confusion_matrix(labels_test, labels_predicted)

O classificador que obteve o maior f1-score foi Gradiente Boosting classifier. Assim, ele será usado para predizer os rótulos do conjunto de teste.

# Classificação

O treinamento será refeito, agora utilizando o conjunto inteiro para treinar o modelo.

In [1]:
labels_train_data = train_data.dep_delayed_15min
train_data.drop(columns="dep_delayed_15min", inplace=True)

clf_boosting.fit(train_data, labels_train_data)

In [1]:
labels_predicted = clf_boosting.predict(test)

In [1]:
labels_submission = pd.Series(labels_predicted)
labels_submission = labels_submission.map(lambda label: 'Y' if label==0 else 'N')
labels_submission.to_csv('submission.csv', index=False)